In [1]:
#Thanks to Luciano Prevedello and Barbaros Selner Erdal.
#https://www.kaggle.com/code/felipekitamura/head-ct-hemorrhage-kernel/notebook
##First of all, we are going to load the images and labels
from glob import glob
import os
import pandas as pd 
import cv2
import numpy as np

# Now we define the dimensions of our images.

img_width, img_height = 128, 128

train_files = sorted(glob('C:\\Users\\charl\\Documents\\College\\Spring 2022\\Thesis\\Data\\Hemorrage\\Train\\*.png'))
train_labels = pd.read_csv('C:\\Users\\charl\\Documents\\College\\Spring 2022\\Thesis\\Data\\Hemorrage\\Train\\labels.csv')[' hemorrhage'].tolist()

test_files = sorted(glob('C:\\Users\\charl\\Documents\\College\\Spring 2022\\Thesis\\Data\\Hemorrage\\Test\\*.png'))
test_labels = pd.read_csv('C:\\Users\\charl\\Documents\\College\\Spring 2022\\Thesis\\Data\\Hemorrage\\Test\\labels.csv')[' hemorrhage'].tolist()

recon_test_files = sorted(glob('C:\\Users\\charl\\Documents\\College\\Spring 2022\\Thesis\\Data\\Hemorrage\\Reconstructed Test\\*.png'))
recon_test_labels = pd.read_csv('C:\\Users\\charl\\Documents\\College\\Spring 2022\\Thesis\\Data\\Hemorrage\\Reconstructed Test\\labels.csv')[' hemorrhage'].tolist()


train_images = np.empty((len(train_files), img_width, img_height))
test_images = np.empty((len(test_files), img_width, img_height))
recon_test_images = np.empty((len(recon_test_files), img_width, img_height))

for i, _file in enumerate(train_files):
    train_images[i, :, :] = cv2.resize(cv2.imread(_file, 0), (img_width, img_height))
for i, _file in enumerate(test_files):
    test_images[i, :, :] = cv2.resize(cv2.imread(_file, 0), (img_width, img_height))
for i, _file in enumerate(recon_test_files):
    recon_test_images[i, :, :] = cv2.resize(cv2.imread(_file, 0), (img_width, img_height))      
    
print ('\033[1m' + 'Ready for next step!')

Ready for next step!


In [2]:
#Now we split the dataset into train (80%), validation (10%) and test (10%) sets.
from sklearn.model_selection import train_test_split
import numpy as np

val_images, test_images, val_labels, test_labels = train_test_split(test_images, test_labels, test_size=0.5, random_state=1)

print((len(train_images), len(val_images), len(test_images)))

(169, 15, 16)


In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as K
import tensorflow as tf
input_shape = (img_width, img_height, 1)

SIIM_custom_model = None

SIIM_custom_model = Sequential()

#Below we have the first Convolutional Layer

SIIM_custom_model.add(Conv2D(32, (3, 3), input_shape=input_shape))
SIIM_custom_model.add(Activation('relu'))

#We then add a MaxPool layer, which will reduce the size of the output of the first conv layer in 75%.
#This is performed to avoid an exagerated increase in the number of parameters of the network.
#Don't worry if you do not understand in detail each one of these operations right now. Try to focus on the big picture.

SIIM_custom_model.add(MaxPooling2D(pool_size=(2, 2)))

#We will add more convolutional layers, followed by MaxPool layers

SIIM_custom_model.add(Conv2D(32, (3, 3)))
SIIM_custom_model.add(Activation('relu'))
SIIM_custom_model.add(MaxPooling2D(pool_size=(2, 2)))

SIIM_custom_model.add(Conv2D(64, (3, 3)))
SIIM_custom_model.add(Activation('relu'))
SIIM_custom_model.add(MaxPooling2D(pool_size=(2, 2)))

#Finally, we will add two dense layers, or 'Fully Connected Layers'.
#These layers are classical neural nets, without convolutions.

SIIM_custom_model.add(Flatten())
SIIM_custom_model.add(Dense(64))
SIIM_custom_model.add(Activation('relu'))

#Dropout is an overfitting reduction technique.

SIIM_custom_model.add(Dropout(0.5))

#Now, we will set the output o the network.
#The Dense function has the argument "1" because the net output is the hematoma x non-hematoma classification

SIIM_custom_model.add(Dense(1))

#The output is either 0 or 1 and this can be obtained with a sigmoid function.

SIIM_custom_model.add(Activation('sigmoid'))

#Let's compile the network.

SIIM_custom_model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

print ('\033[1m' + 'Ready for next step!')

Ready for next step!


In [4]:

import warnings
warnings.filterwarnings('ignore')

nb_train_samples = len(train_images)
nb_validation_samples = len(val_images)
epochs = 100
batch_size = 10


# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.0,
    zoom_range=0.1,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)

# this is the augmentation configuration we will use for validation:
val_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow(
    train_images[..., np.newaxis],
    train_labels,
    batch_size=batch_size)

validation_generator = val_datagen.flow(
    val_images[..., np.newaxis],
    val_labels,
    batch_size=batch_size)

print ('\033[1m' + 'Ready for next step!')

Ready for next step!


In [5]:
history = SIIM_custom_model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/100
16/16 [==============================] - ETA: 0s - loss: 0.7364 - accuracy: 0.49 - 5s 28ms/step - loss: 0.7278 - accuracy: 0.4906 - val_loss: 0.6710 - val_accuracy: 0.9000
Epoch 2/100
16/16 [==============================] - 0s 13ms/step - loss: 0.6877 - accuracy: 0.5975 - val_loss: 0.7023 - val_accuracy: 0.4000
Epoch 3/100
16/16 [==============================] - 0s 13ms/step - loss: 0.6628 - accuracy: 0.6101 - val_loss: 0.8149 - val_accuracy: 0.3000
Epoch 4/100
16/16 [==============================] - 0s 12ms/step - loss: 0.6235 - accuracy: 0.6352 - val_loss: 0.5799 - val_accuracy: 0.8000
Epoch 5/100
16/16 [==============================] - 0s 13ms/step - loss: 0.6206 - accuracy: 0.6667 - val_loss: 0.5140 - val_accuracy: 0.8000
Epoch 6/100
16/16 [==============================] - 0s 12ms/step - loss: 0.6201 - accuracy: 0.6730 - val_loss: 0.4980 - val_accuracy: 0.8000
Epoch 7/100
16/16 [==============================] - 0s 13ms/step - loss: 0.5619 - accuracy: 0.7547 - val_

In [6]:
import matplotlib.pyplot as plt
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

KeyError: 'acc'

In [7]:
#Now we evaluate on the test set. Remember to make pixels between [0, 1] by dividing by 255.
y_preds = []
for index in range(0,len(test_images)):
    y_preds.append(SIIM_custom_model.predict((test_images[index] // 255).reshape(-1, img_height, img_width, 1), test_labels[index]))

m = tf.keras.metrics.BinaryAccuracy()
m.update_state(test_labels, y_preds)
print(f'Result of original test data: {m.result().numpy()}')

for index in range(0,len(test_images)):
    y_preds.append(SIIM_custom_model.predict((recon_test_images[index] // 255).reshape(-1, img_height, img_width, 1), recon_test_labels[index]))

m = tf.keras.metrics.BinaryAccuracy()
m.update_state(recon_test_labels, y_preds)
print(f'Result of reconstructed test data: {m.result().numpy()}')

Result of original test data: 0.75
Result of reconstructed test data: 0.5483871102333069
